In [1]:
import os
import cv2
import json
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
%matplotlib inline

In [2]:
GROUND_TRUTH_PATH = "/home/boomalope2/nerf_estimator/src/blender/test/jackal_evaluation_data/ground_truth.json"
TRANSFORM_PATH = "/home/boomalope2/nerf_estimator/src/blender/test/jackal_evaluation_data/transforms_10.json"

with open(GROUND_TRUTH_PATH, "r") as f:
    ground_truth = json.load(f)
with open(TRANSFORM_PATH, "r") as f:
    transform = json.load(f)

In [3]:
trans = []
rot = []
abs_trans = []
abs_rot = []

for i in range(len(ground_truth["frames"])):
    ground_truth_frame = np.asarray(ground_truth["frames"][i]["transform_matrix"])
    transform_frame = np.asarray(transform["frames"][i]["transform_matrix"])
    
    R_ground_truth = ground_truth_frame[:3, :3]
    R_transform = transform_frame[:3, :3]

    R_relative = np.matmul(R_transform, np.linalg.inv(R_ground_truth))
    r = Rotation.from_matrix(R_relative).as_euler("xyz", degrees=True)

    t_ground_truth = ground_truth_frame[:3, 3]
    t_transform = transform_frame[:3, 3]
    t_relative = t_transform - R_relative @ t_ground_truth
    
    rot.append(r)
    trans.append(t_relative)

    abs_trans.append(np.linalg.norm(t_relative, ord=2))

    trace = R_relative.trace()
    abs_rot.append(np.arccos((trace - 1) / 2) * 180 / math.pi)

In [4]:
print(np.abs(trans).mean(axis=0))
print(np.abs(rot).mean(axis=0))

print(np.abs(abs_trans).mean())
print(np.abs(abs_rot).mean())
print(np.array(abs_rot).mean()*math.pi/180)

[8.04449282e-02 8.03401143e-02 2.73855013e-16]
[2.06855544e-15 1.82351802e-15 8.26595785e+00]
0.12646919523227843
8.265957851893306
0.14426818034661598
